In [3]:
SERVER_PORT=49052#порт сервера
SERVER_HOST='0.0.0.0'#хост сервера
DATABASE_PORT=27017#порт Базы данных СРП

import datetime
from bson.objectid import ObjectId
import random
import string
import hmac
import hashlib
from hashlib import sha256
from flask import Flask
from flask import request,redirect
from flask import Response
from flask import render_template
from os import listdir
from os.path import isfile, join
import os
from pymongo import MongoClient
import pymongo
import os.path
from base64 import b64encode
from collections import OrderedDict
from hashlib import sha256
from hmac import HMAC
from urllib.parse import urlparse, parse_qsl, urlencode
import json

app = Flask(__name__)#создание объекта сервера
app.config['TEMPLATES_AUTO_RELOAD'] = True

@app.after_request#запрет кеширования
def add_header(r):
    r.headers["Cache-Control"] = "no-cache, no-store, must-revalidate"
    r.headers["Pragma"] = "no-cache"
    r.headers["Expires"] = "0"
    r.headers['Cache-Control'] = 'public, max-age=0'
    return r

@app.route('/',methods=["POST"])#проверка команды
def command():
    t=request.json#данные от Клиента СРП
    client = MongoClient(port=DATABASE_PORT)
    db=client.srp
    appID=t["appID"]
    appObject=db.apps.find_one({"_id":ObjectId(request.json["appID"])})#поиск объекта приложения
    tSign=b64encode(HMAC(appObject["secretKey"].encode(), request.json["commandHash"].encode(), sha256).digest())#вычисление подписи
    if request.json["appURL"]!=appObject["appSite"]:#проверка соответствия адреса приложения заявленному
        return "False"#если приложение было запущено по другому адресу, команда не будет выполнена
    if tSign.decode("utf-8")==request.json["serverSign"]:#проверка правильности подписи
        allowedCommands=list()#лист с доступными для приложения командами
        for i in ["list","read","write","delete","mkdir","rename"]:
            if appObject[i]=="on":#команда доступна, если ей соответствует значение on
                allowedCommands.append(i)
        return str(request.json["type"] in allowedCommands)#проверка наличия команды в списке разрешённых
    else:
        return "False"#если подпись не совпала, решение отрицательное

app.run(host=SERVER_HOST, port=SERVER_PORT, threaded=True)#запуск сервера

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:49052/ (Press CTRL+C to quit)
